In [2]:
import os
import sys
import gc
from time import time, sleep

import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import vaex
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg
# pandas on ray has moved to Modin
# import ray.dataframe as rpd

In [7]:
# data based on https://www.kaggle.com/c/ieee-fraud-detection/data
folder = "/home/vaclav/Data/Kaggle/EEE-CIS_Fraud_Detection"
files = ["train_transaction.csv", "train_identity.csv"]
paths = [os.path.join(folder, f) for f in files]

In [4]:
stats = {}

# Pandas

In [8]:
pd.__version__

'1.1.4'

In [9]:
stats["pandas"] = {}
s = stats["pandas"]

ts = time()
df = pd.read_csv(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = pd.read_csv(paths[1])
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

ts = time()
grp = dff["isFraud","ProductCD","card4","card6","id_15","id_31","TransactionAmt"].fillna("")
.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"].agg(["mean","sum"])
te = time()
s["aggregation"] = te-ts

ts = time()
dff.sort_values(by=["card1","addr1","D9"], inplace=True)
dff.sort_values(by=["addr1","D9","card1"], inplace=True)
dff.sort_values(by=["D9","card1","addr1"], inplace=True)
te = time()
s["sorting"] = te-ts

In [38]:
pd.DataFrame(stats)

,pandas
aggregation,0.075788
load_identity,0.682109
load_transactions,18.279765
merge,3.196074
sorting,2.224164


In [37]:
dff.to_pickle("data/dff.pkl")

In [21]:
# Because julia groups by including N\A, let's just check that number of groups matches
grp = dff[["isFraud","ProductCD","card4","card6","id_15","id_31","TransactionAmt"]].fillna("~U~")\
.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"].agg(["mean","sum"])
grp.shape

(4553, 2)

In [39]:
def clean(wait_time: int=15):
    """Cleans created DataFrames and call the garbage collector to actions. Wait for 15s by default"""
    df, df2, dff, grp = None, None, None, None
    gc.collect()
    sleep(wait_time)
    return None

In [40]:
clean()


In [41]:
def list_variables_memory_usage() -> dict:
    """Memory of existing local variables"""
    local_vars = list(locals().items())
    return {var: sys.getsizeof(obj) for var, obj in local_vars}

# Dask
When to use dask - https://docs.dask.org/en/latest/dataframe.html#common-uses-and-anti-uses

In [42]:
stats["dask"] = {}
s = stats["dask"]

ts = time()
df = dd.read_csv(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = dd.read_csv(paths[1])
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

# the difference is that we call compute method, which runs all the computations at this point
ts = time()
grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"]\
    .agg(["mean","sum"])\
    .compute()
te = time()
s["aggregation"] = te-ts

# parallel soring is tricky that is why there are only work arounds in dask. 
ts = time()
dff.set_index("card1").compute()
te = time()
s["sorting"] = te-ts


In [43]:
clean()

In [44]:
pd.DataFrame(stats)

,pandas,dask
load_transactions,18.279765,0.083901
load_identity,0.682109,0.028268
merge,3.196074,0.073891
aggregation,0.075788,20.837958
sorting,2.224164,71.282675


In [45]:
stats["dask_indexed"] = {}
s = stats["dask_indexed"]

ts = time()
df = dd.read_csv(paths[0]).set_index("TransactionID")
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = dd.read_csv(paths[1]).set_index("TransactionID")
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, left_index=True, right_index=True)
te = time()
s["merge"] = te-ts

# the difference is that we call compute method, which runs all the computations at this point
ts = time()
grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"]\
    .agg(["mean","sum"])\
    .compute()
te = time()
s["aggregation"] = te-ts

# parallel soring is tricky that is why there are only work arounds in dask. 
ts = time()
dff.set_index("card1").compute()
te = time()
s["sorting"] = te-ts


In [50]:
clean()
pd.DataFrame(stats)

,pandas,dask,dask_indexed,vaex
load_transactions,18.279765,0.083901,14.930128,NaN
load_identity,0.682109,0.028268,0.761821,NaN
merge,3.196074,0.073891,0.078762,NaN
aggregation,0.075788,20.837958,23.130105,NaN
sorting,2.224164,71.282675,75.393628,NaN


# Vaex

In [97]:
vaex.__version__

{'vaex-core': '2.0.3',
 'vaex-viz': '0.4.0',
 'vaex-hdf5': '0.6.0',
 'vaex-server': '0.3.1',
 'vaex-astro': '0.7.0',
 'vaex-jupyter': '0.5.2',
 'vaex-ml': '0.9.0',
 'vaex-arrow': '0.5.1'}

In [98]:
tool = "vaex"
stats[tool] = {}
s = stats[tool]


ts = time()
df = vaex.open(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = vaex.open(paths[1])
te = time()
s["load_identity"] = te-ts


In [99]:
ts = time()
dff = df.join(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

In [100]:
# the difference is that we call compute method, which runs all the computations at this point
ts = time()
grp = dff.groupby([dff["isFraud"],dff["ProductCD"],dff["card4"],dff["card6"],dff["id_15"],dff["id_31"]], 
                  agg=[vaex.agg.mean('TransactionAmt'), vaex.agg.sum('TransactionAmt')])
te = time()
s["aggregation"] = te-ts


In [111]:
# the difference is that we call compute method, which runs all the computations at this point
ts = time()
dff_s = dff.sort(by=["card1","addr1","D9"])
dff_s = dff.sort(by=["addr1","D9","card1"])
dff_s = dff.sort(by=["D9","card1","addr1"])
te = time()
s["sorting"] = te-ts

In [113]:
pd.DataFrame(stats)

,pandas,dask,dask_indexed,vaex
load_transactions,18.279765,0.083901,14.930128,18.734002
load_identity,0.682109,0.028268,0.761821,1.023915
merge,3.196074,0.073891,0.078762,0.131490
aggregation,0.075788,20.837958,23.130105,0.383996
sorting,2.224164,71.282675,75.393628,1.035000
sort,NaN,NaN,NaN,0.329828


In [24]:
clean()

# PySpark

In [25]:
from pyspark import SparkContext
sc = SparkContext()
sc.version
sc.stop()

In [26]:
# Create my_spark
my_spark = SparkSession.builder \
    .master("local") \
    .appName("Pandas Alternative") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [27]:
tool = "spark"
stats[tool] = {}
s = stats[tool]


ts = time()
df = my_spark.read.csv(paths[0],inferSchema = True,header= True) 
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = my_spark.read.csv(paths[1],inferSchema = True,header= True) 
te = time()
s["load_identity"] = te-ts

In [28]:

ts = time()
dff = df.join(df2, "TransactionID")
te = time()
s["merge"] = te-ts

In [29]:
# the difference is that we call collect method, which runs all the computations at this point
#ts = time()
#grp = dff.groupby([dff["isFraud"],dff["ProductCD"],dff["card4"],dff["card6"],dff["id_15"],dff["id_31"]]) \
#        .agg(avg("TransactionAmt"), sum("TransactionAmt"))\
#        .collect()
#te = time()
#s["aggregation"] = te-ts
#s["all"] = te-tss

In [33]:
# the difference is that we call collect method, which runs all the computations at this point
ts = time()
grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"]) \
        .agg(avg("TransactionAmt"), sum("TransactionAmt"))\
        .collect()
te = time()
s["aggregation"] = te-ts


In [16]:
stats_df = pd.DataFrame(stats)
stats_df.loc['Total'] = stats_df.sum()
stats_df

,pandas
aggregation,0.060114
all,19.908346
load_identity,0.480164
load_transactions,17.354527
merge,2.013150
Total,39.816302


# Modin

In [13]:
mpd.__version__

'0.8.2'

In [15]:
tool = "modin"
stats[tool] = {}
s = stats[tool]


ts = time()
df = mpd.read_csv(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = mpd.read_csv(paths[1])
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

# modin defaults to pandas for multiple column aggregation and then fails on KeyError, though the key is available
ts = time()
try:
    grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"].agg(["mean","sum"])
except Exception as e:
    print(e)
te = time()
s["aggregation"] = te-ts


distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting


KilledWorker: ('lambda-dc847cac2df298f0ded2b3e426e3824d', <Worker 'tcp://127.0.0.1:56445', name: 7, memory: 0, processing: 3>)

In [ ]:
pd.DataFrame(stats)

In [16]:
clean()

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
